# Retrieval Augmented Question (RAG) with Llama3-8B-Instruct on SageMaker JumpStart and Amazon OpenSearch using LangChain

RAG Application use cases with Llama3-8B, BGE Large embedding model on SageMaker and OpenSearch as Vector Database

In this notebook, we demonstrate the use of [Llama3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) text generation combined with [BGE Large En v1.5](https://huggingface.co/BAAI/bge-large-en-v1.5) embedding model to efficiently construct a Retrieval Augmented Generation (RAG) QnA system on a SageMaker Notebook. This notebook, powered by an `ml.t3.medium instance`, enables the deployment of LLMs on [SageMaker JumpStart](https://aws.amazon.com/sagemaker/jumpstart/). These can be called with an API endpoint created by SageMaker, which we then use to build, experiment with, and tune for comparing Advanced RAG application techniques using [LangChain](https://www.langchain.com/). Additionally, we showcase how we can use [OpenSearch](https://aws.amazon.com/opensearch-service/) Vector Engine Embedding store to archive and retrieve embeddings, integrating it into your RAG workflow. 

![Architecture](RAG-OpenSearch.png)

## Prerequisites

---
This Jupyter Notebook can be run on a t3.medium instance (ml.t3.medium). However, to deploy `Llama3-8B-Instruct` and `BGE Large En v1.5` models, you may need to request a quota increase. 

To request a quota increase, follow these steps:

1. Navigate to the [Service Quotas console](https://console.aws.amazon.com/servicequotas/).
2. Choose Amazon SageMaker.
3. Review your default quota for the following resources:
   - `ml.g5.2xlarge` for endpoint usage. You will need two instances. 
4. If needed, request a quota increase for these resources.

### Changing instance type
---
Models are supported on the following instance types:

 - Llama3-8B Text Generation: `ml.g5.2xlarge`, `ml.g5.4xlarge`, `ml.g5.8xlarge`, `ml.g5.12xlarge`, `ml.g5.24xlarge`, `ml.g5.48xlarge`, and `ml.p4d.24xlarge`
 - BGE Large En v1.5: `ml.g5.2xlarge`, `ml.c6i.xlarge`,`ml.g5.4xlarge`, `ml.g5.8xlarge`, `ml.p3.2xlarge`, and `ml.g4dn.2xlarge`

By default, the JumpStartModel class selects a default instance type available in your region. If you would like to use a different instance type, you can do so by specifying instance type in the JumpStartModel class.

`my_model = JumpStartModel(model_id=model_id, instance_type="ml.g5.12xlarge")`

## Contents
---

1. [Requirements](#Requirements)
2. [Model Deployment](#Model-Deployment)
3. [Setup LangChain](#Setup-LangChain)
4. [Data Preparation](#Data-Preparation)
5. [Question Answering with LangChain Vector Store Wrapper](#Question-Answering-with-LangChain-Vector-Store-Wrapper)
6. [Conclusion](#Conclusion)
7. [Clean Up Resources](#Clean-Up-Resources)

## Requirements
---

1. Create an Amazon SageMaker Notebook Instance - [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html)
    - For Notebook Instance type, choose `ml.t3.medium`.
2. For Select Kernel, choose [conda_python3](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-prepare.html).
3. Install the required packages.

<div class="alert alert-block alert-info"> 

<b>NOTE:

- </b> For <a href="https://aws.amazon.com/sagemaker/studio/" target="_blank">Amazon SageMaker Studio</a>, select Kernel "<span style="color:green;">Python 3 (ipykernel)</span>".

- For <a href="https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html" target="_blank">Amazon SageMaker Studio Classic</a>, select Image "<span style="color:green;">Base Python 3.0</span>" and Kernel "<span style="color:green;">Python 3</span>".

</div>

To run this notebook you would need to install the following dependencies:

In [43]:
%%writefile requirements.txt
langchain==0.1.14
pypdf==4.1.0
opensearch-py==2.8.0
requests-aws4auth==1.3.1
boto3==1.34.58
sqlalchemy==2.0.29
deprecated==1.2.15

Overwriting requirements.txt


In [44]:
!pip install -U -r requirements.txt --quiet

if you see this error → ERROR: pip's dependency resolver does not currently take into account all the packages after pip install you can ignore it

<div class="alert alert-block alert-warning"> 

<b>NOTE:</b>

Before proceeding, please verify that you have the correct version of the SQLAlchemy library installed. This notebook requires SQLAlchemy >= 2.0.0.

To check your installed SQLAlchemy version, you can run the following code:

```python
import sqlalchemy
print(sqlalchemy.__version__)
```

If the version displayed is less than 2.0.0, and you have already installed the correct version using `pip`, you may need to "<span style="color:green;">restart</span>" or "<span style="color:green;">shutdown</span>" the Jupyter Notebook kernel to load the updated library.

To restart the kernel, go to the "Kernel" menu and select "Restart Kernel". If that doesn't work, try shutting down the notebook completely and relaunching it.

Restarting or shutting down the kernel will resolve any dependency issues and ensure that the correct SQLAlchemy version is loaded.

If you haven't installed SQLAlchemy >= 2.0.0 yet, you can do so by running the following command in your terminal or command prompt:

```
pip install sqlalchemy>=2.0.29
```

Once the installation is complete, restart or shutdown the Jupyter Notebook kernel as described above.

</div>

In [45]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.29


In [46]:
import langchain
print(langchain.__version__)

0.1.14


In [47]:
try:
    import sagemaker
except ImportError:
    !pip install sagemaker --quiet

## Model Deployment
---

Deploy `Llama 3 8B Instruct` LLM model on Amazon SageMaker JumpStart:

In [48]:
# Import the JumpStartModel class from the SageMaker JumpStart library
from sagemaker.jumpstart.model import JumpStartModel

In [49]:
# Specify the model ID for the HuggingFace Llama 3 8b Instruct LLM model
model_id = "meta-textgeneration-llama-3-8b-instruct"
accept_eula = True
model = JumpStartModel(model_id=model_id, instance_type="ml.g5.4xlarge")
llm_predictor = model.deploy(accept_eula=accept_eula)

Model 'meta-textgeneration-llama-3-8b-instruct' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-2.s3.us-east-2.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.


[04/14/25 05:38:04] INFO     Model 'meta-textgeneration-llama-3-8b-instruct' requires accepting        ]8;id=984461;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/jumpstart/utils.py\utils.py]8;;\:]8;id=317476;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/jumpstart/utils.py#597\597]8;;\
                             end-user license agreement (EULA). See                                                
                             https://jumpstart-cache-prod-us-east-2.s3.us-east-2.amazonaws.com/fmhMeta             
                             data/eula/llama3Eula.txt for terms of use.                                            

                    INFO     Creating model with name:                                              ]8;id=739703;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=944996;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             meta-textgeneration-llama-3-8b-instruct-2025-04-14-05-38-04-840                       

[04/14/25 05:38:05] INFO     Creating endpoint-config with name                                     ]8;id=197789;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=395685;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             meta-textgeneration-llama-3-8b-instruct-2025-04-14-05-38-04-923                       

[04/14/25 05:38:06] INFO     Creating endpoint with name                                            ]8;id=901048;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=756401;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\
                             meta-textgeneration-llama-3-8b-instruct-2025-04-14-05-38-04-923                       

----------!

Deploy `BGE Large En` embedding model on Amazon SageMaker JumpStart:

In [50]:
# Specify the model ID for the HuggingFace BGE Large EN Embedding model
model_id = "huggingface-sentencesimilarity-bge-large-en-v1-5"
text_embedding_model = JumpStartModel(model_id=model_id)
embedding_predictor = text_embedding_model.deploy()

No instance type selected for inference hosting endpoint. Defaulting to ml.g5.2xlarge.


[04/14/25 05:44:42] INFO     No instance type selected for inference hosting endpoint. Defaulting to   ]8;id=391119;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/jumpstart/factory/model.py\model.py]8;;\:]8;id=823484;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/jumpstart/factory/model.py#238\238]8;;\
                             ml.g5.2xlarge.                                                                        

                    INFO     Creating model with name:                                              ]8;id=538188;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=861182;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\
                             hf-sentencesimilarity-bge-large-en-v1-5-2025-04-14-05-44-42-790                       

[04/14/25 05:44:43] INFO     Creating endpoint-config with name                                     ]8;id=765757;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=558133;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\
                             hf-sentencesimilarity-bge-large-en-v1-5-2025-04-14-05-44-42-872                       

[04/14/25 05:44:44] INFO     Creating endpoint with name                                            ]8;id=488198;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=329141;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\
                             hf-sentencesimilarity-bge-large-en-v1-5-2025-04-14-05-44-42-872                       

--------!

## Setup LangChain
---

In [51]:
import json
import sagemaker

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

Get endpoint names from predictors.

In [52]:
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name
# predictor = model.deploy(accept_eula=accept_eula)
llm_endpoint_name = llm_predictor.endpoint_name
embedding_endpoint_name = embedding_predictor.endpoint_name

Transform input and output data to proccess API calls for`Llama 3 8B Instruct` on Amazon SageMaker

In [53]:
from typing import Dict

class Llama38BContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 1000,
                "top_p": 0.9,
                "temperature": 0.6,
                "stop": ["<|eot_id|>"],
            },
        }
        input_str = json.dumps(
            payload,
        )
        #print(input_str)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        #print(response_json)
        content = response_json["generated_text"].strip()
        return content

Instantiate the LLM with SageMaker and LangChain

In [54]:
# Instantiate the content handler for Llama3-8B
llama_content_handler = Llama38BContentHandler()

# Setup for using the Llama3-8B model with SageMaker Endpoint
llm = SagemakerEndpoint(
     endpoint_name=llm_endpoint_name,
     region_name=region, 
     model_kwargs={"max_new_tokens": 1024, "top_p": 0.9, "temperature": 0.7},
     content_handler=llama_content_handler
 )

[04/14/25 05:49:58] INFO     Found credentials from IAM Role:                                   ]8;id=405231;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=152457;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Transform input and output data to proccess API calls for`BGE Large En` on Amazon SageMaker

In [55]:
from typing import List

class BGEContentHandlerV15(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            text_inputs (list[str]): A list of input text strings to be processed.
            model_kwargs (Dict): Additional keyword arguments to be passed to the endpoint.
               Possible keys and their descriptions:
               - mode (str): Inference method. Valid modes are 'embedding', 'nn_corpus', and 'nn_train_data'.
               - corpus (str): Corpus for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - top_k (int): Top K for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - queries (list[str]): Queries for Nearest Neighbor. Required when mode is 'nn_corpus' or 'nn_train_data'.
        Returns:
            The transformed bytes input.
        """
        input_str = json.dumps(
            {
                "text_inputs": text_inputs,
                **model_kwargs
            }
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]

Instantiate the embedding model with SageMaker and LangChain

In [56]:
bge_content_handler = BGEContentHandlerV15()
sagemaker_embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=embedding_endpoint_name,
    region_name=region,
    model_kwargs={"mode": "embedding"},
    content_handler=bge_content_handler,
)

[04/14/25 05:50:13] INFO     Found credentials from IAM Role:                                   ]8;id=898511;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=810954;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## Data Preparation
---

Let's first download some of the files to build our document store.

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on.

In [57]:
!mkdir -p ./data

from urllib.request import urlretrieve
urls = [
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/c7c14359-36fa-40c3-b3ca-5bf7f3fa0b96.pdf',
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/d2fde7ee-05f7-419d-9ce8-186de4c96e25.pdf',
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/f965e5c3-fded-45d3-bbdb-f750f156dcc9.pdf',
    'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001018724/336d8745-ea82-40a5-9acc-1a89df23d0f3.pdf'
]

filenames = [
    'AMZN-2024-10-K-Annual-Report.pdf',
    'AMZN-2023-10-K-Annual-Report.pdf',
    'AMZN-2022-10-K-Annual-Report.pdf',
    'AMZN-2021-10-K-Annual-Report.pdf'
]

metadata = [
    dict(year=2024, source=filenames[0]),
    dict(year=2023, source=filenames[1]),
    dict(year=2022, source=filenames[2]),
    dict(year=2021, source=filenames[3])]

data_root = "./data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    urlretrieve(url, file_path)

If you take a look into the Amazon 10-Ks, the first 4 pages are all the very similar and may skew the responses if you they are kept in the embeddings. This will cause repetition, take longer to generate embeddings, and may skew your results. In the next section you will take the downloaded data, trim the 10-K (first 4 pages) and overwrite them as processed files.

In [58]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

# Iterate over each PDF file
for idx, local_pdf in enumerate(local_pdfs):
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    
    if idx == 0:
        # Keep the first 4 pages for the first document
        for pagenum in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[pagenum]
            pdf_writer.add_page(page)
    else:
        # Remove the first 4 pages for other documents
        for pagenum in range(4, len(pdf_reader.pages)):
            page = pdf_reader.pages[pagenum]
            pdf_writer.add_page(page)

    # Write the modified content to a new file
    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

After downloading we can load the documents with the help of [DirectoryLoader from PyPDF available under LangChain](https://python.langchain.com/en/latest/reference/modules/document_loaders.html) and splitting them into smaller chunks.

Note: The retrieved document/text should be large enough to contain enough information to answer a question; but small enough to fit into the LLM prompt. Also the embeddings model has a limit of the length of input tokens limited to 512 tokens, which roughly translates to ~2000 characters. For the sake of this use-case we are creating chunks of roughly 1000 characters with an overlap of 100 characters using [RecursiveCharacterTextSplitter](https://python.langchain.com/en/latest/modules/indexes/text_splitters/examples/recursive_text_splitter.html).

In [59]:
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]

    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
)

docs = text_splitter.split_documents(documents)
print(docs[100])

page_content='Year Ended December 31,\n 2022 2023\nCash provided by (used in):\nOperating activities $ 46,752 $ 84,946 \nInvesting activities (37,601) (49,833)\nFinancing activities 9,718 (15,879)\nOur principal sources of liquidity are cash flows generated from operations and our cash, cash equivalents, and marketable securities balances, which, at\nfair value, were $70.0 billion and $86.8 billion as of December 31, 2022 and 2023. Amounts held in foreign currencies were $18.3 billion and $23.5 billion as\nof December 31, 2022 and 2023. Our foreign currency balances include British Pounds, Canadian Dollars, Euros, Indian Rupees, and Japanese Yen.\nCash provided by (used in) operating activities was $46.8 billion and $84.9 billion in 2022 and 2023. Our operating cash flows result primarily from cash\nreceived from our consumer, seller, developer, enterprise, and content creator customers, and advertisers, offset by cash payments we make for products and' metadata={'year': 2024, 'source'

Before we are proceeding we are looking into some interesting statistics regarding the document preprocessing we just performed:

In [60]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)

print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')

Average length among 437 documents loaded is 3347 characters.
After the split we have 1801 documents as opposed to the original 437.
Average length among 1801 documents (after split) is 822 characters.


We had 4 PDF documents which have been split into smaller ~500 chunks.

Now we can see how a sample embedding would look like for one of those chunks.

In [61]:
sample_embedding = np.array(sagemaker_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.0014751  -0.03200122 -0.02893505 ... -0.03613092  0.01975137
 -0.00402734]
Size of the embedding:  (1024,)


We can use [OpenSearch](https://aws.amazon.com/opensearch-service/) implementation with [LangChain](https://python.langchain.com/v0.2/api_reference/community/vectorstores/langchain_community.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html) to ingest the documents to OpenSearch service.

In [62]:
##Provide the Opensearch url here 
## Retrieve OpenSearch url by going to AWS Console->Amazon OpenSearch Service-> Click on the Domain you would liek to use. 
import os
aos_url = "vpc-opensearchservi-csq91xifctnw-wzia6jszaiwua2owoo4ufo2jwq.us-east-2.es.amazonaws.com" 
os.environ['OPENSEARCH_URL'] = aos_url
region

'us-east-2'

In [63]:
# Create a Boto3 session
import boto3
session = boto3.Session()

# Get the account id
account_id = boto3.client('sts').get_caller_identity().get('Account')

# Get the current region
region = session.region_name

cfn = boto3.client('cloudformation')

# Method to obtain output variables from Cloudformation stack. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## Setup variables to use for the rest of the demo
cloudformation_stack_name = "rag-opensearch"

outputs = get_cfn_outputs(cloudformation_stack_name)

# We will just print all the variables so you can easily copy if needed.
# outputs

In [70]:
# Connect to OpenSearch using the internal username and password obtained from AWS Secrets Manager
from opensearchpy import OpenSearch, RequestsHttpConnection

kms = boto3.client('secretsmanager')
aos_username = outputs['OpenSearchUsername']
aos_password = kms.get_secret_value(SecretId=outputs['OpenSearchPasswordArn'])
auth = (aos_username, aos_password)
# print(auth)
#"admin","*****" )
    
# Create OpenSearch client
aos_client = OpenSearch(
    hosts=[aos_url],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)
# print(f"Connected to OpenSearch endpoint :{aos_client}")


In [71]:
# Define the role mapping to grant permissions to  AOS username and notebook_iam_role_arn
from sagemaker import get_execution_role


sagemaker_execution_role = get_execution_role()
print(sagemaker_execution_role)
role_name = "all_access"

role_mapping = {
    "backend_roles": [sagemaker_execution_role],
    "users" : [ aos_username]
}


# Create the role mapping
response = aos_client.security.create_role_mapping(role=role_name, body=role_mapping)
print("Role mapping created:", response)

arn:aws:iam::043632497353:role/rag-opensearch-NBRole-D7pElYPoydQM


[04/14/25 06:09:24] WARNING  PUT                                                                        ]8;id=302228;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/opensearchpy/connection/base.py\base.py]8;;\:]8;id=537790;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/opensearchpy/connection/base.py#280\280]8;;\
                             https://vpc-opensearchservi-csq91xifctnw-wzia6jszaiwua2owoo4ufo2jwq.us-eas            
                             t-2.es.amazonaws.com:9200/_plugins/_security/api/rolesmapping/all_access              
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/u            
                             rllib3/connection.py", line 198, in _new_conn                                         
                                 sock = connection.create_connection(                                              
                               File                                                                                
                             "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/u            
                             rllib3/util/connection.py", line 85, in create_connection                             
                                 raise err                                                                         
                               File                                                                                
                             "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/u            
                             rllib3/util/connection.py", line 73, in create_connection                             
                                 sock.connect(sa)                                                                  
                             TimeoutError: timed out                                                               
                                                                                                                   
                             The above exception was the direct cause of the following exception:                  
                                                                                                                   
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/u            
                             rllib3/connectionpool.py", line 787, in urlopen                                       
                                 response = self._make_request(                                                    
                               File                                                                                
                             "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/u            
                             rllib3/connectionpool.py", line 488, in _make_request                                 
                                 raise new_e                                                                       
                               File                                                                                
                             "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/u            
                             rllib3/connectionpool.py", line 464, in _make_request                                 
                  

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/connection.py:19 │
│ 8 in _new_conn                                                                                   │
│                                                                                                  │
│    195 │   │   :return: New socket connection.                                                   │
│    196 │   │   """                                                                               │
│    197 │   │   try:                                                                              │
│ ❱  198 │   │   │   sock = connection.create_connection(                                          │
│    199 │   │   │   │   (self._dns_host, self.port),                                              │
│    200 │   │   │   │   self.timeout,                                                             │
│    201 │   │   │   │   source_address=self.source_address,                                       │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/util/connection. │
│ py:85 in create_connection                                                                       │
│                                                                                                  │
│    82 │                                                                                          │
│    83 │   if err is not None:                                                                    │
│    84 │   │   try:                                                                               │
│ ❱  85 │   │   │   raise err                                                                      │
│    86 │   │   finally:                                                                           │
│    87 │   │   │   # Break explicitly a reference cycle                                           │
│    88 │   │   │   err = None                                                                     │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/util/connection. │
│ py:73 in create_connection                                                                       │
│                                                                                                  │
│    70 │   │   │   │   sock.settimeout(timeout)                                                   │
│    71 │   │   │   if source_address:                                                             │
│    72 │   │   │   │   sock.bind(source_address)                                                  │
│ ❱  73 │   │   │   sock.connect(sa)                                                               │
│    74 │   │   │   # Break explicitly a reference cycle                                           │
│    75 │   │   │   err = None                                                                     │
│    76 │   │   │   return sock                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TimeoutError: timed out

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/urllib3/connectionpool.p │
│ y:787 in urlopen                                                                                 │
│                                                                                                  │
│    784 │   │   │   response_conn = conn if not release_conn else 

In [ ]:
from langchain_community.vectorstores import OpenSearchVectorSearch
from opensearchpy import RequestsHttpConnection
from requests_aws4auth import AWS4Auth
import boto3

##Make sure the execution role has permissions for Opensearch
service = "es"
region_4Auth = region
credentials = boto3.Session().get_credentials()
# print(credentials.access_key)
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    region_4Auth,
    service,
    session_token=credentials.token
)




In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

# Initialize OpenSearchVectorSearch
vectorstore_opensearch = OpenSearchVectorSearch.from_documents(
    docs,
    sagemaker_embeddings,
    http_auth=awsauth,  # Auth will use the IAM role
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    bulk_size=2000  # Increase this to accommodate the number of documents you have
)

# Wrap the OpenSearch vector store with the VectorStoreIndexWrapper
wrapper_store_opensearch = VectorStoreIndexWrapper(vectorstore=vectorstore_opensearch)


## Question Answering with LangChain Vector Store Wrapper
---

We use the wrapper provided by LangChain which wraps around the Vector Store and takes input the LLM. This wrapper performs the following steps behind the scences:

- Takes input the question
- Create question embedding
- Fetch relevant documents
- Stuff the documents and the question into a prompt
- Invoke the model with the prompt and generate the answer in a human readable manner.

*Note: In this example we are using `Llama 3 8B Instruct` as the LLM under Amazon SageMaker, this particular model performs best if the inputs are provided under `<|begin_of_text|><|start_header_id|>system<|end_header_id|>`, `{{system_message}}`, `<|eot_id|><|start_header_id|>user<|end_header_id|>`, `{{user_message}}`, and the model is requested to generate an output after `<|eot_id|><|start_header_id|>assistant<|end_header_id|>`. In the cell below you see an example of how to control the prompt such that the LLM stays grounded and doesn't answer outside the context.*

In [ ]:
prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant.
<|eot_id|><|start_header_id|>user<|end_header_id|>
{query}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["query"]
)

In [ ]:
query = "How did AWS perform in 2021?"

In [ ]:
answer = wrapper_store_opensearch.query(question=PROMPT.format(query=query), llm=llm)
print(answer)

We can ask another question.

In [ ]:
query_2 = "How much square footage did Amazon have in North America in 2023?"

In [ ]:
answer = wrapper_store_opensearch.query(question=PROMPT.format(query=query_2), llm=llm)
print(answer)

### Regular Retriever Chain
---
In the above scenario you explored the quick and easy way to get a context-aware answer to your question. Now let's have a look at a more customizable option with the help of [RetrievalQA](https://docs.smith.langchain.com/cookbook/hub-examples/retrieval-qa-chain) where you can customize how the documents fetched should be added to prompt using `chain_type` parameter. Also, if you want to control how many relevant documents should be retrieved then change the `k` parameter in the cell below to see different outputs. In many scenarios you might want to know which were the source documents that the LLM used to generate the answer, you can get those documents in the output using `return_source_documents` which returns the documents that are added to the context of the LLM prompt. `RetrievalQA` also allows you to provide a custom [prompt template](https://python.langchain.com/docs/modules/model_io/prompts/quick_start/) which can be specific to the model.

In [ ]:
from langchain.chains import RetrievalQA

prompt_template = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

This is a conversation between an AI assistant and a Human.

<|eot_id|><|start_header_id|>user<|end_header_id|>

Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
#### Context ####
{context}
#### End of Context ####

Question: {question}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore_opensearch.as_retriever(
        search_type="similarity", search_kwargs={"k": 3}
    ),
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

Let's start asking questions:

In [ ]:
query = "How did AWS perform in 2023?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "What are some of the risk factors associated to Amazon?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "Was Amazon involved in any lawsuits in 2022? What were they?"
result = qa({"query": query})
print(result['result'])

print(f"\n{result['source_documents']}")

In [ ]:
query = "What was Amazon's revenue in 2021?"

result = qa({"query": query})

print(result['result'])

print(f"\n{result['source_documents']}")

## Conclusion
---

Congratulations on completing the Retrieval Augmented Generation(RAG) notebook with `Llama3 8b`! Through this notebook, you were able to learn how to leverage the power of `Llama3 8b` with  `LangChain` and `OpenSearch`

In the above implementation of Advanced RAG based Question Answering we have explored the following concepts and how to implement them using Amazon SageMaker JumpStart and it's LangChain integration.

- Deploying models on Amazon SageMaker JumpStart
- Setting up `Llama3-8b` and `BGE Large En v1.5` with LangChain
- Loading documents of different kind and generating embeddings to create a vector store
- Retrieving documents to the question using the following approaches from LangChain
    - Regular Retrieval Chain
- Preparing a prompt which goes as input to the LLM
- Present an answer in a human friendly manner

### Take-aways
---
- Experiment with different retrieval techniques
- Leverage `Llama3-8b` and `BGE Large En v1.5` models available under Amazon SageMaker JumpStart
- Explore options such as persistent storage of embeddings and document chunks
- Integration with enterprise data stores

## Clean Up Resources
---

In [ ]:
# Delete resources
llm_predictor.delete_model()
llm_predictor.delete_endpoint()
embedding_predictor.delete_model()
embedding_predictor.delete_endpoint()

# Thank You!